### Generate Sample Dataset for Data Cleaning Project

In [20]:
import pandas as pd
import random
import numpy as np

In [22]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Set parameters for number of rows
num_records = 2000

In [24]:
# Helper functions
def random_name():
    first_names = ["John", "Jane", "Alex", "Emily", "Chris", "Taylor", "Jordan", "Morgan", "Casey", "Drew"]
    last_names = ["Smith", "Johnson", "Brown", "Williams", "Jones", "Miller", "Davis", "Garcia", "Martinez", "Hernandez"]
    return f"{random.choice(first_names)} {random.choice(last_names)}"

def random_diagnosis_code():
    return f"{random.choice(['A', 'B', 'C', 'D'])}{random.randint(10, 99)}.{random.randint(0, 9)}"

def random_date():
    start_date = pd.Timestamp("2023-01-01")
    end_date = pd.Timestamp("2024-11-01")
    return start_date + (end_date - start_date) * random.random()

In [26]:
# Generate dataset
data = {
    "ID": [random.randint(1000, 9999) for _ in range(num_records)],
    "Name": [random_name() for _ in range(num_records)],
    "Age": [random.randint(0, 120) if random.random() > 0.1 else np.nan for _ in range(num_records)],  # Some unrealistic ages
    "Date of Service": [random_date() for _ in range(num_records)],
    "Diagnosis Code": [random_diagnosis_code() if random.random() > 0.05 else np.nan for _ in range(num_records)],  # 5% missing
    "Total Cost": [round(random.uniform(-500, 15000), 2) if random.random() > 0.02 else np.nan for _ in range(num_records)],  # Some negative costs
    "Gender": [random.choice(["Male", "Female", np.nan, "Unknown"]) for _ in range(num_records)],  # Includes 'Unknown' and NaN
    "Phone": [f"+1-{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}" if random.random() > 0.1 else np.nan for _ in range(num_records)],
    "Email": [f"user{random.randint(1, 2000)}@example.com" if random.random() > 0.05 else "invalid_email" for _ in range(num_records)],  # Some invalid emails
    "Street": [f"{random.randint(100, 999)} Main St" for _ in range(num_records)],
    "City": [random.choice(["New York", "Los Angeles", "Chicago", np.nan]) for _ in range(num_records)],  # Includes NaN
    "State": [random.choice(["NY", "CA", "IL", "TX", "FL", "INVALID"]) for _ in range(num_records)],  # Includes 'INVALID'
    "Zip Code": [str(random.randint(10000, 99999)) for _ in range(num_records)],
    "Insurance": [random.choice(["Yes", "No", ""]) for _ in range(num_records)],  # Includes empty strings
    "Notes": [random.choice(["", "Follow-up required", "Check-up", ""] * 5) for _ in range(num_records)]  # Includes empty strings
}

In [28]:
# Convert to DataFrame
mock_df = pd.DataFrame(data)

# Save dataset to CSV
file_path = "c:\dataset\python\sample_data.csv"
mock_df.to_csv(file_path, index=False)
print(f"Sample dataset created and saved as {file_path}")

Sample dataset created and saved as c:\dataset\python\sample_data.csv


In [30]:
print(mock_df.head())

     ID             Name   Age     Date of Service Diagnosis Code  Total Cost  \
0  2824  Morgan Martinez   7.0 2023-05-05 09:03:34          A40.7    10444.03   
1  1409     Casey Garcia  38.0 2023-02-09 12:25:53          A54.1     1638.53   
2  5506    Alex Martinez  86.0 2023-05-03 10:17:19          A80.7     8832.91   
3  5012     Morgan Jones  68.0 2024-05-23 15:22:05          C15.1      996.87   
4  4657    Emily Johnson   NaN 2023-06-23 18:22:43            NaN    12439.18   

  Gender            Phone                 Email       Street         City  \
0    NaN  +1-722-318-8145  user1251@example.com  505 Main St  Los Angeles   
1    NaN  +1-622-477-9042   user279@example.com  799 Main St          NaN   
2   Male  +1-980-538-4072  user1191@example.com  371 Main St  Los Angeles   
3   Male  +1-344-667-2305   user437@example.com  522 Main St      Chicago   
4    NaN              NaN  user1824@example.com  867 Main St     New York   

  State Zip Code Insurance               Notes  
0

### Data Cleaning

In [35]:
import pandas as pd
import numpy as np

In [37]:
# load dataset
file_path = "c:\dataset\python\sample_data.csv"
df = pd.read_csv(file_path)

In [39]:
# Find all duplicates based on all columns
duplicates = df[df.duplicated(keep=False)]
# Group by all column to get the count of each duplicate row
duplicates_counts = duplicates.groupby(list(duplicates.columns)).size().reset_index(name='count')
# print(duplicates_counts)
df = df.drop_duplicates()

In [41]:
# Sort by ID (ascending) & Date of Service (descending) to keep that latest row

# Convert Date of Service to datetime format
df['Date of Service'] = pd.to_datetime(df['Date of Service'])
# sort the DataFrame by ID(ascending) & Date of Service(descending) descending order
df = df.sort_values(by=['ID', 'Date of Service'], ascending=[True, False])

In [43]:
# Find duplicates based on ID and Name
duplicates_columns = df[df.duplicated(subset=['ID','Name'], keep=False)]
# Group by the specifiied columns to get the count of each duplcate 
duplicates_counts = duplicates_columns.groupby(['ID', 'Name']).size().reset_index(name='count')
# print(duplicates_counts)
df = df.drop_duplicates(subset=['ID', 'Name'])

In [45]:
# Verify  Data
filtered_rows = df[df['ID'] == 2215]
print(filtered_rows)

        ID        Name   Age     Date of Service Diagnosis Code  Total Cost  \
1349  2215  John Smith  71.0 2024-09-12 19:27:27          C67.2    12932.31   

     Gender            Phone                Email       Street         City  \
1349    NaN  +1-367-708-3519  user745@example.com  695 Main St  Los Angeles   

     State  Zip Code Insurance Notes  
1349    TX     67126       Yes   NaN  


In [47]:
# Fill missing values for categorical data
df["Gender"].fillna("Unknown", inplace=True)
df["City"].fillna("Unknown", inplace=True)
df["Diagnosis Code"].fillna("Unknown", inplace=True)
df["Insurance"].fillna("No", inplace=True)

In [49]:
# Drop a record with unrealistic or missing ages
df = df[(df["Age"] >= 0) & (df["Age"] <= 120)]

In [51]:
# Verify date
filtered_rows = df[df['Age'] > 120]
print(filtered_rows)

Empty DataFrame
Columns: [ID, Name, Age, Date of Service, Diagnosis Code, Total Cost, Gender, Phone, Email, Street, City, State, Zip Code, Insurance, Notes]
Index: []


In [53]:
# Replace invalid or negative costs with null value
df["Total Cost"] = df["Total Cost"].apply(lambda x: np.nan if x < 0 else x)

In [55]:
# Verify data
df = df.sort_values(by="Total Cost", ascending=True)
print(df.head())

        ID              Name   Age     Date of Service Diagnosis Code  \
1670  4515   Morgan Williams  25.0 2023-05-10 20:58:52          D51.9   
1306  8138        Alex Smith  92.0 2024-06-06 14:51:56          B95.8   
633   7209      Morgan Brown  53.0 2024-01-28 23:54:29          B50.8   
1937  5715    Casey Williams  77.0 2023-05-09 20:07:00          C43.4   
1731  4694  Jordan Hernandez  75.0 2023-08-26 14:29:10          A43.1   

      Total Cost   Gender            Phone                 Email       Street  \
1670        1.92   Female  +1-620-404-2187    user63@example.com  836 Main St   
1306        4.20     Male  +1-597-177-5389  user1981@example.com  303 Main St   
633         6.38     Male  +1-711-412-7725  user1628@example.com  585 Main St   
1937       20.15  Unknown              NaN  user1499@example.com  902 Main St   
1731       36.22   Female  +1-925-517-7810         invalid_email  554 Main St   

             City State  Zip Code Insurance               Notes  
1670    

In [57]:
# Group by State with its counts
df["State"] = df["State"].str.strip()
state_counts = df["State"].value_counts()
print(state_counts)

State
TX         314
IL         305
INVALID    302
FL         301
NY         291
CA         291
Name: count, dtype: int64


In [59]:
# If state not in valid_states, insert null value
valid_states = ["IL", "CA", "TX", "NY", "FL"]
df["State"] = df["State"].apply(lambda x: x if x in valid_states else np.nan)

In [61]:
# Validate email address contains "@", "." in address
df["Email"] = df["Email"].apply(lambda x: x if "@" in x and "." in x.split("@")[-1] else np.nan)

In [63]:
# Validare phone number NOT NULL and start with +1- to capture US Phone Number
df["Phone"] = df["Phone"].apply(lambda x: x if pd.notnull(x) and x.startswith("+1-") else np.nan)

In [65]:
# Fill missing Total Cost with the median value
df["Total Cost"].fillna(df["Total Cost"].median(), inplace=True)

In [67]:
# Stripe whitespace from notes
df["Notes"] = df["Notes"].str.strip()

In [69]:
# Save cleaned dataset
cleaned_file_path = "c:\dataset\python\cleaned_data.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved a {cleaned_file_path}")

Cleaned dataset saved a c:\dataset\python\cleaned_data.csv
